[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/googlecolab/colabtools/blob/master/notebooks/colab-github-demo.ipynb)

# Initialization

For our analysis, we have used Python Version 3.8.12 on Anaconda. Our environment involved the use of the following libraries:   
- pandas 1.4.2  
- pycaret 2.3.6, which was installed using the `pip install pycaret[full]` command (see [pycaret installation guidelines for more details](https://pycaret.gitbook.io/docs/get-started/installation)). To run the code on Google Colab, please uncomment the first line of code below. 
- imblearn 0.7.0  
- sklearn 0.23.2  
- statsmodels 0.13.2   

Note that the packages above can all be installed using the `pip install pycaret[full]` command since they are dependencies for the pycaret package. **Note that the version number of the packages used on Google Colab for facilitating the use of our code is somewhat different from when we ran the code on our local machines.**

## Required Libraries and Reading the Data

In [67]:
!pip install markupsafe==2.0.1 # (run first time you run the file)
# !pip install jinja2 # (first time you run the file)
# !pip install pycaret  # (first time you run the file)

import jinja2
from google.colab import drive 
drive.mount('/content/drive')

from pandas.api.types import CategoricalDtype
import pandas as pd

from pycaret.classification import *
import imblearn as im
import sklearn

import statsmodels.api as sm

# reading the data
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/tavr_paper/TAVRDataApp.csv')
df.columns = df.columns.str.lower()


SyntaxError: ignored

In [ ]:
df.head() # printing the first five rows of the data

,id,died,age,female,race,elective,aweekend,zipinc_qrtl,hosp_region,hosp_division,...,prior_mi,prior_pci,prior_ppm,prior_tia_stroke,pulmonary_circulation_disorder,smoker,valvular_disease,weight_loss,endovascular_tavr,transapical_tavr
0,1,No,67,Female,White,Elective,No,ThirdQ,Northeast,New England,...,No,No,No,No,No,No,Yes,No,Yes,No
1,2,No,90,Male,White,Elective,No,ThirdQ,Northeast,New England,...,No,No,No,No,No,No,Yes,No,Yes,No
2,3,No,90,Male,White,NonElective,No,FourthQ,Northeast,New England,...,Yes,Yes,No,No,Yes,Yes,Yes,No,Yes,No
3,4,No,80,Female,White,NonElective,No,FirstQ,Northeast,New England,...,No,No,No,No,No,Yes,Yes,No,No,Yes
4,5,No,84,Female,White,Elective,No,FourthQ,Northeast,New England,...,No,No,Yes,No,No,No,Yes,No,No,Yes


## Transforming the Data

In the code chunk below, we transformed all strings to categorical data. Additionally, we have transformed the zip quartile variable to be ordinal. 

In [ ]:
# converting all columns of type object to categorical
df.loc[:, df.dtypes == 'object'] =\
    df.select_dtypes(['object'])\
    .apply(lambda x: x.astype('category'))

# converting ordinal column to ordinal
ordinal_cat = CategoricalDtype(categories = ['FirstQ', 'SecondQ', 'ThirdQ', 'FourthQ'], ordered = True)
df.zipinc_qrtl = df.zipinc_qrtl.astype(ordinal_cat)
display( df.zipinc_qrtl.unique() )

['ThirdQ', 'FourthQ', 'FirstQ', 'SecondQ']
Categories (4, object): ['FirstQ' < 'SecondQ' < 'ThirdQ' < 'FourthQ']

---

# The Machine Learning Setup

In the code chunk below, we import the method for random over sampling from the `imbalanced-learn` library, and then define the setup for our machine learning experiment. In this markdown, we only show the setup for the `l2 reguralized logistic regression model` which have deployed into our application. For more information on the models examined for research questions 1 and 2, please refer to their corresponding Jupyter Notebooks. Then we imported functions to compute both the specificity and balanced accuracy from the `imbalanced-learn` and `sklearn` libraries, respectively. Note that the API for the `setup function` can be found at [Classification -pycaret](https://pycaret.readthedocs.io/en/latest/api/classification.html). 

In [ ]:
up_sample = im.over_sampling.RandomOverSampler(random_state = 2022, sampling_strategy='minority')

up_setup = setup(df, target = 'died', train_size = 0.8, preprocess = True, 
                 imputation_type = 'iterative', categorical_imputation = 'constant',
                 ordinal_features = { 'zipinc_qrtl' : ['FirstQ', 'SecondQ', 'ThirdQ', 'FourthQ'] },
                 ignore_features = ['id'], 
                 handle_unknown_categorical = True, fix_imbalance=True, 
                 fix_imbalance_method = up_sample, fold_strategy = 'stratifiedkfold',
                 fold = 10, n_jobs = -1,
                 session_id=2022, experiment_name='tavr_up', log_experiment=True,
                 normalize = True, normalize_method = 'minmax',
                 feature_selection= True, remove_multicollinearity= True)

# adding specificity and balanced accuracy to the computed metrics
add_metric('Spec', 'Spec.', im.metrics.specificity_score)
add_metric('Balanced Accuracy', 'Balanced Accuracy', sklearn.metrics.balanced_accuracy_score)


,Description,Value
0,session_id,2022
1,Target,died
2,Target Type,Binary
3,Label Encoded,"No: 0, Yes: 1"
4,Original Data,"(54739, 47)"
5,Missing Values,False
6,Numeric Features,1
7,Categorical Features,44
8,Ordinal Features,True
9,High Cardinality Features,False


2022/05/27 13:02:17 INFO mlflow.tracking.fluent: Experiment with name 'tavr_up' does not exist. Creating a new experiment.


Name                                                 Balanced Accuracy
Display Name                                         Balanced Accuracy
Score Function       <function balanced_accuracy_score at 0x7f63a52...
Scorer                            make_scorer(balanced_accuracy_score)
Target                                                            pred
Args                                                                {}
Greater is Better                                                 True
Multiclass                                                        True
Custom                                                            True
Name: Balanced_Accuracy, dtype: object

## Training the l2 Reguarlized Logistic Regression Model

Using the `create_model()` pycaret function, we trained the reguarlized l2 logistic regression model using our 10-fold cross validation. The model is trained on 80% of the  training data per the `setup()`. Note that the LogisticRegression() model is imported from sklearn. The code chunk below presents the results for the 10-fold cross validation.

In [ ]:
lr = create_model('lr')


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Spec.,Balanced Accuracy
Fold,,,,,,,,,
0,0.7637,0.8030,0.7191,0.0596,0.1101,0.0754,0.1586,0.7646,0.7419
1,0.7714,0.7990,0.6818,0.0581,0.1070,0.0727,0.1504,0.7732,0.7275
2,0.7773,0.8232,0.7273,0.0631,0.1160,0.0821,0.1682,0.7784,0.7528
3,0.7691,0.7901,0.6705,0.0567,0.1045,0.0701,0.1456,0.7711,0.7208
4,0.7659,0.7823,0.6705,0.0559,0.1032,0.0687,0.1438,0.7679,0.7192
5,0.7668,0.8357,0.7614,0.0628,0.1160,0.0819,0.1727,0.7670,0.7642
6,0.7710,0.7854,0.6629,0.0572,0.1053,0.0705,0.1450,0.7732,0.7181
7,0.7700,0.7691,0.6180,0.0535,0.0985,0.0634,0.1302,0.7732,0.6956
8,0.7719,0.8167,0.6742,0.0583,0.1072,0.0725,0.1491,0.7739,0.7240


## Tuning the Trained l2 Reguarlized Logistic Regression Model

We have chosen to optimize the model based on the 'Balanced Accuracy' metric since we wanted to ensure that we have a good balance between sensitivity and specificity. The model below is tuned based on 10 combinations of the hyperparameters using the `scikit-learn` libraries default grid. For more information on the `tune_model()` parameters, please see [pycaret classification API documents](https://pycaret.readthedocs.io/en/latest/api/classification.html). 

In [ ]:
tuned_lr = tune_model(lr, optimize = 'Balanced Accuracy', n_iter = 10)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Spec.,Balanced Accuracy
Fold,,,,,,,,,
0,0.7637,0.8030,0.7191,0.0596,0.1101,0.0754,0.1586,0.7646,0.7419
1,0.7714,0.7990,0.6818,0.0581,0.1070,0.0727,0.1504,0.7732,0.7275
2,0.7771,0.8233,0.7273,0.0630,0.1159,0.0820,0.1680,0.7781,0.7527
3,0.7696,0.7900,0.6818,0.0576,0.1063,0.0719,0.1494,0.7714,0.7266
4,0.7659,0.7823,0.6705,0.0559,0.1032,0.0687,0.1438,0.7679,0.7192
5,0.7664,0.8357,0.7614,0.0627,0.1158,0.0817,0.1724,0.7665,0.7639
6,0.7710,0.7854,0.6629,0.0572,0.1053,0.0705,0.1450,0.7732,0.7181
7,0.7703,0.7691,0.6180,0.0536,0.0986,0.0635,0.1304,0.7734,0.6957
8,0.7716,0.8167,0.6742,0.0582,0.1071,0.0724,0.1489,0.7737,0.7239


## Plotting the Model

### Interactive Plots

In [ ]:
evaluate_model(tuned_lr)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

### Threshold

From the code chunk below and its associated plot, we can see that the optimal threshold for the logistic regression based on the different metrics should be around the default value of 0.5. Recall/Sensitivity can be significantly improved with smaller probability thresholds; however, this is associated with a significant drop in both the accuracy and specificity. Also note that both the specificity and accuracy result have very similar curves, which is expected due to the imbalanced nature of our data.

In [ ]:
best_lr_threshold = optimize_threshold(tuned_lr, optimize = 'AUC')

---

# Holdout Performance

Based on the output below, we see that the model's performance metrics are quite similar to those observed on the validation data. Hence, we have no indication of over fitting.

In [ ]:
pred_holdout = predict_model(tuned_lr)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Spec.,Balanced Accuracy
0,Logistic Regression,0.7695,0.8122,0.7105,0.0618,0.1138,0.0785,0.1611,0.7707,0.7406


---

# Finalizing the Model

The final model model is retrained over our entire data (i.e., by merging both the training and testing datasets together). This is the model that we save and use in our app.

In [ ]:
final_model = finalize_model(tuned_lr)
save_model(final_model, '/content/drive/My Drive/Colab Notebooks/tavr_paper/final_model')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=None,
          steps=[('dtypes',
                  DataTypes_Auto_infer(categorical_features=[],
                                       display_types=True,
                                       features_todrop=['id'], id_columns=[],
                                       ml_usecase='classification',
                                       numerical_features=[], target='died',
                                       time_features=[])),
                 ('imputer',
                  Iterative_Imputer(add_indicator=False,
                                    classifier=LGBMClassifier(boosting_type='gbdt',
                                                              class_weight=None,
                                                              colsample_bytr...
                                        target_variable='died', threshold=0.9)),
                 ('dfs', 'passthrough'), ('pca', 'passthrough'),
                 ['trained_model',
                  LogisticRegres

---

# Creating the App

## Example data for the app

In [ ]:
ex_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/tavr_paper/example_data.csv')
ex_data = ex_data.to_numpy()
ex_data = ex_data.tolist()

## Testing the data for the app

In [ ]:
temp_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/tavr_paper/example_data2.csv')
temp_df.columns = temp_df.columns.str.lower()

temp_df.loc[:, temp_df.dtypes == 'object'] =\
    temp_df.select_dtypes(['object'])\
    .apply(lambda x: x.astype('category'))

 # converting ordinal column to ordinal
temp_df.zipinc_qrtl = temp_df.zipinc_qrtl.astype(ordinal_cat)

pred = predict_model(final_model, temp_df, raw_score=True)
pred

print({'Death %': pred['Score_Yes'][0].astype('float64'),
       'Survival %': pred['Score_No'][0].astype('float64' ),
       'Predicted Death Outcome:': pred['Label'][0]})

{'Death %': 0.689, 'Survival %': 0.311, 'Predicted Death Outcome:': 'Yes'}


## Setting up Gradio

In [ ]:
import gradio as gr
import numpy as np

import io
import pickle
import requests
import urllib.request
import shutil

url = 'https://raw.githubusercontent.com/fmegahed/tavr_paper/main/data/example_data2.csv'
download = requests.get(url).content

ex_data =pd.read_csv(io.StringIO(download.decode('utf-8')))
ex_data = ex_data.to_numpy()
ex_data = ex_data.tolist()


def predict(age, female, race, elective, aweekend, zipinc_qrtl, hosp_region, hosp_division, hosp_locteach,
            hosp_bedsize, h_contrl, pay, anemia, atrial_fibrillation, 
            cancer, cardiac_arrhythmias, carotid_artery_disease, 
            chronic_kidney_disease, chronic_pulmonary_disease, coagulopathy,
            depression, diabetes_mellitus, drug_abuse, dyslipidemia, endocarditis,
            family_history, fluid_and_electrolyte_disorder, heart_failure,
            hypertension, known_cad, liver_disease, obesity, peripheral_vascular_disease,
            prior_cabg, prior_icd, prior_mi, prior_pci, prior_ppm, prior_tia_stroke,
            pulmonary_circulation_disorder, smoker, valvular_disease, weight_loss,
            endovascular_tavr, transapical_tavr):
  
 

  df = pd.DataFrame.from_dict({
      'age': [age], 'female': [female], 'race': [race], 'elective': elective,
       'aweekend': [aweekend], 'zipinc_qrtl': [zipinc_qrtl], 
       'hosp_region': [hosp_region], 'hosp_division': [hosp_division],
       'hosp_locteach': [hosp_locteach], 'hosp_bedsize': [hosp_bedsize],
       'h_contrl': [h_contrl], 'pay': [pay], 'anemia': [anemia], 
       'atrial_fibrillation': [atrial_fibrillation], 'cancer': [cancer],
       'cardiac_arrhythmias': [cardiac_arrhythmias], 
       'carotid_artery_disease': [carotid_artery_disease], 
       'chronic_kidney_disease': [chronic_kidney_disease], 
       'chronic_pulmonary_disease': [chronic_pulmonary_disease], 
       'coagulopathy': [coagulopathy], 'depression': [depression],
       'diabetes_mellitus': [diabetes_mellitus], 'drug_abuse': [drug_abuse], 
       'dyslipidemia': [dyslipidemia], 'endocarditis': [endocarditis],
       'family_history': [family_history], 'fluid_and_electrolyte_disorder': [fluid_and_electrolyte_disorder],
       'heart_failure': [heart_failure], 'hypertension': [hypertension],
       'known_cad': [known_cad], 'liver_disease': [liver_disease],
       'obesity': [obesity], 'peripheral_vascular_disease': [peripheral_vascular_disease],
       'prior_cabg': [prior_cabg], 'prior_icd': [prior_icd], 'prior_mi': [prior_mi],
       'prior_pci': [prior_pci], 'prior_ppm': [prior_ppm], 'prior_tia_stroke': [prior_tia_stroke],
       'pulmonary_circulation_disorder': [pulmonary_circulation_disorder], 
       'smoker': [smoker], 'valvular_disease': [valvular_disease],
       'weight_loss': [weight_loss], 'endovascular_tavr': [endovascular_tavr],
       'transapical_tavr': [transapical_tavr]
  })
  
  df.loc[:, df.dtypes == 'object'] =\
    df.select_dtypes(['object'])\
    .apply(lambda x: x.astype('category'))

  # converting ordinal column to ordinal
  ordinal_cat = CategoricalDtype(categories = ['FirstQ', 'SecondQ', 'ThirdQ', 'FourthQ'], ordered = True)
  df.zipinc_qrtl = df.zipinc_qrtl.astype(ordinal_cat)

  with urllib.request.urlopen('https://github.com/fmegahed/tavr_paper/blob/main/data/final_model.pkl?raw=true') as response, open('final_model.pkl', 'wb') as out_file:
    shutil.copyfileobj(response, out_file)

  model = load_model('final_model')

  pred = predict_model(model, df, raw_score=True)
  
  return {'Death %': round(100*pred['Score_Yes'][0], 2),
       'Survival %': round(100*pred['Score_No'][0], 2),
       'Predicting Death Outcome:': pred['Label'][0]}

# Defining the containers for each input
age = gr.inputs.Slider(minimum=0, maximum=100, default=60, label="Age")
female = gr.inputs.Dropdown(choices=["Female", "Male"],label = 'Sex')
race = gr.inputs.Dropdown(choices=['Asian or Pacific Islander', 'Black', 'Hispanic', 'Native American', 'White',  'Other'], label = 'Race')
elective = gr.inputs.Radio(choices=['Elective', 'NonElective'], label = 'Elective')
aweekend = gr.inputs.Radio(choices=["No", "Yes"], label = 'Weekend')
zipinc_qrtl = gr.inputs.Radio(choices=['FirstQ', 'SecondQ', 'ThirdQ', 'FourthQ'], label = 'Zip Income Quartile')
hosp_region = gr.inputs.Radio(choices=['Midwest', 'Northeast', 'South', 'West'], label = 'Hospital Region')
hosp_division = gr.inputs.Radio(choices=['New England', 'Middle Atlantic', 'East North Central', 'West North Central', 'South Atlantic', 'East South Central', 'West South Central', 'Mountain', 'Pacific'], label = 'Hospital Division')
hosp_locteach = gr.inputs.Radio(choices=['Urban teaching', 'Urban nonteaching', 'Rural'], label= 'Hospital Location/Teaching')
hosp_bedsize = gr.inputs.Radio(choices=['Small', 'Medium', 'Large'], label= 'Hospital Bedsize')
h_contrl = gr.inputs.Radio(choices= ['Government_nonfederal', 'Private_invest_own', 'Private_not_profit'], label = 'Hospital Control')
pay = gr.inputs.Dropdown(choices= ['Private insurance', 'Medicare', 'Medicaid',  'Self-pay', 'No charge', 'Other'], label = 'Payee')
anemia = gr.inputs.Radio(choices=["No", "Yes"], label = 'Anemia')
atrial_fibrillation = gr.inputs.Radio(choices=["No", "Yes"], label = 'Atrial Fibrillation')
cancer = gr.inputs.Radio(choices=["No", "Yes"], label = 'Cancer')
cardiac_arrhythmias = gr.inputs.Radio(choices=["No", "Yes"], label = 'Cardiac Arrhythmias')
carotid_artery_disease = gr.inputs.Radio(choices=["No", "Yes"], label = 'Carotid Artery Disease') 
chronic_kidney_disease = gr.inputs.Radio(choices=["No", "Yes"], label = 'Chronic Kidney Disease')
chronic_pulmonary_disease = gr.inputs.Radio(choices=["No", "Yes"], label = 'Chronic Pulmonary Disease') 
coagulopathy =  gr.inputs.Radio(choices=["No", "Yes"], label = 'Coagulopathy')
depression = gr.inputs.Radio(choices=["No", "Yes"], label = 'Depression')
diabetes_mellitus = gr.inputs.Radio(choices=["No", "Yes"], label = 'Diabetes Mellitus')
drug_abuse = gr.inputs.Radio(choices=["No", "Yes"], label = 'Drug Abuse')
dyslipidemia = gr.inputs.Radio(choices=["No", "Yes"], label = 'Dyslipidemia')
endocarditis = gr.inputs.Radio(choices=["No", "Yes"], label = 'Endocarditis')
family_history = gr.inputs.Radio(choices=["No", "Yes"], label = 'Family History')
fluid_and_electrolyte_disorder = gr.inputs.Radio(choices=["No", "Yes"], label = 'Fluid and Electrolyte Disorder')
heart_failure = gr.inputs.Radio(choices=["No", "Yes"], label = 'Heart Failure')
hypertension = gr.inputs.Radio(choices=["No", "Yes"], label = 'Hypertension')
known_cad = gr.inputs.Radio(choices=["No", "Yes"], label = 'Known CAD')
liver_disease = gr.inputs.Radio(choices=["No", "Yes"], label = 'Liver Disease')
obesity = gr.inputs.Radio(choices=["No", "Yes"], label = 'Obesity')
peripheral_vascular_disease = gr.inputs.Radio(choices=["No", "Yes"], label = 'Peripheral Vascular Disease')
prior_cabg = gr.inputs.Radio(choices=["No", "Yes"], label = 'Prior CABG')
prior_icd = gr.inputs.Radio(choices=["No", "Yes"], label = 'Prior ICD')
prior_mi = gr.inputs.Radio(choices=["No", "Yes"], label = 'Prior MI')
prior_pci = gr.inputs.Radio(choices=["No", "Yes"], label = 'Prior PCI') 
prior_ppm = gr.inputs.Radio(choices=["No", "Yes"], label = 'Prior PPM')
prior_tia_stroke = gr.inputs.Radio(choices=["No", "Yes"], label = 'Prior TIA Stroke')
pulmonary_circulation_disorder = gr.inputs.Radio(choices=["No", "Yes"], label = 'Pulmonary Circulation Disorder') 
smoker = gr.inputs.Radio(choices=["No", "Yes"], label = 'Smoker')
valvular_disease = gr.inputs.Radio(choices=["No", "Yes"], label = 'Valvular Disease') 
weight_loss = gr.inputs.Radio(choices=["No", "Yes"], label = 'Weight Loss')
endovascular_tavr = gr.inputs.Radio(choices=["No", "Yes"], label = 'Endovascular TAVR')
transapical_tavr = gr.inputs.Radio(choices=["No", "Yes"], label = 'Transapical TAVR', default= 'Yes')


# Defining and launching the interface
gr.Interface(predict, [age, female, race, elective, aweekend, zipinc_qrtl, hosp_region, hosp_division, hosp_locteach,
            hosp_bedsize, h_contrl, pay, anemia, atrial_fibrillation, 
            cancer, cardiac_arrhythmias, carotid_artery_disease, 
            chronic_kidney_disease, chronic_pulmonary_disease, coagulopathy,
            depression, diabetes_mellitus, drug_abuse, dyslipidemia, endocarditis,
            family_history, fluid_and_electrolyte_disorder, heart_failure,
            hypertension, known_cad, liver_disease, obesity, peripheral_vascular_disease,
            prior_cabg, prior_icd, prior_mi, prior_pci, prior_ppm, prior_tia_stroke,
            pulmonary_circulation_disorder, smoker, valvular_disease, weight_loss,
            endovascular_tavr, transapical_tavr], 
            'text',
            live=True,
            title = "Predicting In-Hospital Mortality After TAVR Using Preoperative Variables and Penalized Logistic Regression",
            description = "The app below utilizes the finalized logistic regression model with an l2 penalty based on the manuscript by Alhwiti et al. The manuscript will be submitted to JACC: Cardiovascular Interventions. The data used for model building is all TAVR procedures between 2012 and 2019 as reported in the HCUP NIS database. <br><br> The purpose of the app is to provide evidence-based clinical support for interventional cardiology. <br> <br> For instruction on how to use the app and the encoding required for the variables,  please see <b>XYZ: insert website link here</b>.",
            css = 'https://bootswatch.com/5/journal/bootstrap.css').launch(debug = True);

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://41726.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


Transformation Pipeline and Model Successfully Loaded
Keyboard interruption in main thread... closing server.


## Permanent Link for the App

The app is hosted permanently at: 